In [1]:
#Importing Necessary Libraries

#Data Manipulation/Calculation Libraries
import pandas as pd
import numpy as np
import math

#Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

#NLP 
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
#from fuzzywuzzy import fuzz
from nltk import word_tokenize
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
#from tqdm import tqdm_notebook

#ML Models
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
#from sklearn.preprocessing import Imputer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv("FINAL_v1.csv")
del data['Unnamed: 0']
data.head()

,album_title,artist,track_title,lyrics
0,Girl Code,City Girls,What We Doin',"(The 90's), Yup, haha, House super clean (Clea..."
1,Girl Code,City Girls,Season (Ft. Lil Baby),"It's that season (Y'all know what time it is),..."
2,Girl Code,City Girls,Broke Boy,"If you a broke boy, stay the fuck out my way, ..."
3,Girl Code,City Girls,Clout Chasin',"Yung Miami, ho, JT on the track, Told that lil..."
4,Girl Code,City Girls,Intro (#FREEJT),"You have pre-paid call, You will not be charge..."


In [3]:
data.lyrics = data.lyrics.astype(str)

In [4]:
data['lyrics'][0]

"(The 90's), Yup, haha, House super clean (Clean), fridge on full (Full), Baby with the grandmom, bitch I'm good, We could party all night, where we goin'? What we doin'?, Party all night, where we goin'? What we doin'?, House super clean (Clean), fridge on full (Full), Baby with the grandmom, bitch I'm good, We could party all night, where we goin'? What we doin'?, Party all night, where we goin'? What we doin'?, Girl, it's Friday night, We ain't kicked it in a while on a Friday night, It's time to step out on this Friday night, You actin' like an old lady, Girl, you probably right, Let me look up in the closet, see what I could wear, Check bank account, lemme see what I could spare, Baby daddy might trip, but I don't even care, You don't ever go out, he been out all year, Girl, you gon' get a bitch beat up, Child, please, what time we gon' meet up?, His soft ass ain't gon' do shit, But talk like he always do, bitch, You ain't nothin' but trouble, sho' is, You my dawg, and I love you,

In [5]:
def clean_text(text):
    #Convert to lower case
    text = text.lower()
    #Lemmatizing the text
    lemma = WordNetLemmatizer()
    normalized = " ".join(lemma.lemmatize(word, pos = "v") for word in text.split())
    #Removing White spaces
    normalized = normalized.replace('\d+', '')
    normalized = normalized.strip()
    #Tokenize and extract words that are alpha-numeric
    tokens = word_tokenize(normalized)
    cleaned = [word for word in tokens if word.isalpha()]
    #Create a dictionary of stem-words such as "at" and 
    #"the" that don't contribute to meaning and remove them from the list
    stop_words = set(stopwords.words('english'))
    words = [w for w in cleaned if not w in stop_words]
    return words

In [6]:
data['lyrics_clean'] = [clean_text(text) for text in data["lyrics"]]

In [7]:
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 500)
data[0:1000]

,album_title,artist,track_title,lyrics,lyrics_clean
0,Girl Code,City Girls,What We Doin',"(The 90's), Yup, haha, House super clean (Clea...","[yup, haha, house, super, clean, clean, fridge..."
1,Girl Code,City Girls,Season (Ft. Lil Baby),"It's that season (Y'all know what time it is),...","[season, know, time, niggas, cut, check, reaso..."
2,Girl Code,City Girls,Broke Boy,"If you a broke boy, stay the fuck out my way, ...","[break, boy, stay, fuck, way, niggas, lyin, di..."
3,Girl Code,City Girls,Clout Chasin',"Yung Miami, ho, JT on the track, Told that lil...","[yung, miami, ho, jt, track, tell, lil, bitch,..."
4,Girl Code,City Girls,Intro (#FREEJT),"You have pre-paid call, You will not be charge...","[call, charge, call, call, jatavia, inmate, fe..."
5,Girl Code,City Girls,Drip,"Drip-drip-drip on a hundred, Y'all hoes hate, ...","[hundred, hoe, hate, hundred, hoe, hate, hundr..."
6,Girl Code,City Girls,Trap Star,"I got that million-dollar pussy (Yeah), And a ...","[get, pussy, yeah, nigga, yeah, get, budget, y..."
7,Girl Code,City Girls,Give It a Try (Ft. Jacquees),"Ohh, ahh, Ohh, ahh, Ohh, ahh, You better never...","[ohh, ahh, ohh, ahh, ohh, ahh, better, never, ..."
8,Girl Code,City Girls,Twerk (Ft. Cardi B),"(I want a slim, fine woman with some twerk wit...","[want, slim, fine, woman, twerk, throw, twerk,..."
9,Girl Code,City Girls,Panties and Bra,"When you come to the spot (Come to the spot), ...","[come, spot, come, spot, takin, takin, open, d..."


In [8]:
spotify = pd.read_csv("spotify_selected_data.csv")
del spotify['Unnamed: 0']
spotify.rename(columns = {'album':'album_title', 'name':'track_title'}, inplace = True) 
spotify.head()

,album_title,track_number,id,track_title,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,Girl Code,1,4Vw3GouohCgu8hgGYdl8m9,Intro (#FREEJT),spotify:track:4Vw3GouohCgu8hgGYdl8m9,0.12600,0.664,0.637,0.000000,0.378,-4.597,0.4170,98.892,0.0393,34
1,Girl Code,2,0RRm4OS5ymfZryXBuj0G2m,Twerk (feat. Cardi B),spotify:track:0RRm4OS5ymfZryXBuj0G2m,0.12100,0.788,0.842,0.001120,0.166,-4.006,0.1420,190.077,0.4810,70
2,Girl Code,3,3A2yGHWIzmGEIolwonU69h,Act Up,spotify:track:3A2yGHWIzmGEIolwonU69h,0.01670,0.938,0.638,0.000000,0.111,-4.713,0.1890,97.075,0.3130,75
3,Girl Code,4,6gqvCJJY8Sh4opRxXVO7A7,Season (feat. Lil Baby),spotify:track:6gqvCJJY8Sh4opRxXVO7A7,0.00964,0.594,0.695,0.000000,0.157,-4.634,0.3110,138.833,0.4080,50
4,Girl Code,5,3E7jehnxsH4xZAuxDQg4zC,Broke Boy,spotify:track:3E7jehnxsH4xZAuxDQg4zC,0.00397,0.782,0.690,0.000003,0.175,-3.677,0.0682,102.997,0.3170,41


In [9]:
data['track_title'].replace({'Ft.': 'feat.'}, inplace=True, regex=True)

In [10]:
#merged = pd.merge(data, spotify, on = ['track_title', 'album_title'])

In [11]:
#merged.shape

In [ ]:
#data.to_csv("subset_lyrics.csv")

In [ ]:
#spotify.to_csv("subset_spotify.csv")

In [18]:
merged_subset = pd.read_csv("subset_merged.csv")
del merged_subset['Unnamed: 0']
del merged_subset['X1.x']
del merged_subset['X1.y']
merged_subset.head()

,album_title,track_title,track_number,id,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,artist,lyrics,lyrics_clean
0,17,Ayala (Outro),11,5xbrhx2tMMcN68IT3cJbhS,spotify:track:5xbrhx2tMMcN68IT3cJbhS,0.737,0.570,0.2850,0.013300,0.108,-14.125,0.0381,83.485,0.0973,65,XXXTENTACION,"Ooh, oh-oh, oh-oh, She showed me fake love, ca...","['ooh', 'show', 'fake', 'love', 'ca', 'forget'..."
1,17,Carry On,9,2yZax79pOrYuyIFVW2cZY2,spotify:track:2yZax79pOrYuyIFVW2cZY2,0.768,0.735,0.1970,0.000276,0.296,-13.378,0.5370,147.899,0.3830,76,XXXTENTACION,"How did you get here?, I'm drunk and confused,...","['get', 'drink', 'confused', 'try', 'patient',..."
2,17,Dead Inside (Interlude),7,2vQwlW8H1jFveGNXwZfJCV,spotify:track:2vQwlW8H1jFveGNXwZfJCV,0.973,0.461,0.0279,0.002260,0.169,-21.992,0.0412,88.388,0.3540,65,XXXTENTACION,"Dead inside, Spend a lot of time stuck in this...","['dead', 'inside', 'spend', 'lot', 'time', 'st..."
3,17,Depression & Obsession,3,2H3wWrnO758y0fPH7Ilerg,spotify:track:2H3wWrnO758y0fPH7Ilerg,0.585,0.769,0.1560,0.015000,0.102,-10.700,0.0350,105.950,0.5270,73,XXXTENTACION,"Na, na, na, na, na, Na, na, na, na, na, na, Na...","['na', 'na', 'na', 'na', 'na', 'na', 'na', 'na..."
4,17,Jocelyn Flores,2,7m9OqQk4RVRkw9JJdeAw96,spotify:track:7m9OqQk4RVRkw9JJdeAw96,0.469,0.872,0.3910,0.000004,0.297,-9.144,0.2420,134.021,0.4370,87,XXXTENTACION,"I know you so well, so well, I mean, I can do ...","['know', 'well', 'well', 'mean', 'anything', '..."
